Checks the coverage on created partitions

In [1]:
import json

with open("../../results/partition-creation/stats_part_dbpedia2015_30k.json") as file:
    created = json.load(file)["families"]

partitions = [",".join(fam["predicateSet"]) for fam in created]

In [2]:
encoded_partitions = set()
rev_predicate_map = { predicate: int(index) for predicate, index in (line.split("\t") for line in open(f"../../results/queryshapes/usewod_predicate_map.tsv").readlines())}
for partition in partitions:
    predicates = partition.split(',')
    if -1 in [rev_predicate_map.get(pred, -1) for pred in predicates]:
        print('not defined')
    indices = ",".join([str(rev_predicate_map.get(pred, -1)) for pred in predicates])
    encoded_partitions.add(indices)

not defined
not defined
not defined
not defined


In [3]:
encoded_partitions

{'13117,21802,1083,1726,909,1728,2738,496,79,755,1084,788,396',
 '187,571,502,503',
 '83,119,15,360,1900,13117,1083,58,1726,1727,909,1728,59,1729,57,2738,79,755,1084,788,396,271,11,12',
 '326,914,2747,4,6,253,3377,555',
 '208,104,2069,19157,1326,421,1076,23,214',
 '83,119,15,360,1900,21802,1083,58,1726,1727,909,1728,59,1729,272,57,2738,496,79,755,788,396,271,11,12',
 '326,914,1540,106,2747,4,2069,6,253,246,420,19385,3377,555',
 '83,360,1900,13117,21802,1083,58,1726,909,1728,59,272,2738,496,788,396,271,11,12',
 '2775,144,547,2072,571,1370,2188',
 '547,18105',
 '83,119,15,13117,21802,1083,1726,909,1728,57,2738,79,755,271,11,12',
 '664,326,914,106,757,49,208,6,1326,421,1076,23',
 '83,119,2738',
 '208,2082,246,3056',
 '74,75,106,208,104,1326,421,555,23',
 '21802,1083,58,1726,1727,909,1728,59,1729,57,755,271,11,12',
 '664,326,914,1540,757,247,4,2069,6,253,246,555,214',
 '74,106,2747,247,104,246,420,1176,214',
 '83,15,13117,2738,496,79',
 '254,187,502',
 '83,119,15,360,1900,21802,1726,1728,5

In [4]:
import json
import pandas as pd
import importlib
import matplotlib.pyplot as plt
import utils.minimum_union as mu

parts = [2013, 2014, 2015, 2016]
usewod_df, usewod_meta = mu.import_combined([f"../../results/queryshapes/usewod_{part}" for part in parts])

In [5]:
covered_weight = usewod_df.apply(lambda row: row["weight"] if row["set"].issubset(encoded_partitions) else 0, axis=1).sum()

In [6]:
covered_weight / usewod_meta["VALID_QUERIES"]

0.021263460716037214

In [7]:
import json
with open("../../results/minimum-unions/usewod_minimum_unions.json", "r") as file:
    usewod_min_unions_json = json.load(file)
usewod_min_unions_70 = usewod_min_unions_json[-3]

In [8]:
complex_partitions = [p for p in usewod_min_unions_70["optimalPartitions"] if len(p.split(",")) > 1]
complex_partitions_set = set(complex_partitions)
covered_weight_complex = usewod_df.apply(lambda row: row["weight"] if row["set"].issubset(complex_partitions_set) else 0, axis=1).sum()
covered_weight_complex / usewod_meta["VALID_QUERIES"]

0.21533700109422424

In [9]:
complex_partitions

['2,8',
 '5,8',
 '8,11,12',
 '11,12',
 '8,10',
 '8,13',
 '8,14',
 '8,15',
 '8,16',
 '8,9',
 '5,31',
 '5,14,17,18',
 '5,14,17,19',
 '5,14,18,20',
 '5,14,18,21',
 '5,14,19,20',
 '5,14,19,21',
 '5,17,18',
 '5,17,18,22',
 '5,17,19',
 '5,17,19,22',
 '5,18,20',
 '5,18,20,22',
 '5,18,21',
 '5,18,21,22',
 '5,19,20',
 '5,19,20,22',
 '5,19,21',
 '5,19,21,22',
 '5,8,17,18',
 '5,8,17,19',
 '5,8,18,20',
 '5,8,18,21',
 '5,8,19,20',
 '5,8,19,21',
 '8,52',
 '5,58,59,909,1083',
 '5,58,59,909,1083,1726',
 '5,58,59,909,1083,1727',
 '5,58,59,909,1083,1728',
 '5,58,59,909,1083,1729',
 '3,4,326',
 '3,5',
 '2,8,16',
 '8,38',
 '71,129',
 '5,8,51',
 '1,8',
 '3,4,5,7',
 '8,31,44',
 '1,8,11,12,83,170',
 '5,24',
 '14,44,71',
 '45,677',
 '8,31',
 '5,24,63',
 '5,8,11,12,1077',
 '2,5',
 '5,100',
 '5,502',
 '5,98',
 '3,4,5,6',
 '8,43',
 '45,129',
 '4,8,16,101,102',
 '3,4,5,7,8',
 '3,4,5,8',
 '3,5,7,8,136',
 '3,5,8,136',
 '3,4,5,6,8',
 '3,5,6,8,136',
 '5,43',
 '2,3,13,316',
 '5,129',
 '1,2,8',
 '31,129',
 '8,83',
 '15

In [10]:
resolved_partitions = []
predicate_map = { int(index): predicate for predicate, index in (line.split("\t") for line in open(f"../../results/queryshapes/usewod_predicate_map.tsv").readlines())}
for partition in complex_partitions:
    indices = partition.split(',')
    predicates = ",".join([predicate_map[int(index)] for index in indices])
    resolved_partitions.append(predicates)

In [11]:
with open("../../results/queryshapes/usewod_partitions_complex_only.txt", "w") as file:
    for part in resolved_partitions:
        file.write(part + "\n")

## 2016 only

In [12]:
import json
with open("../../results/minimum-unions/usewod_2016_minimum_unions.json", "r") as file:
    usewod_min_unions_2016 = json.load(file)
usewod_min_unions_2016_68 = usewod_min_unions_2016[-3]

In [13]:
parts = [2016]
usewod_df_16, usewod_meta_16 = mu.import_combined([f"../../results/queryshapes/usewod_{part}" for part in parts])

In [14]:
complex_partitions_16 = [p for p in usewod_min_unions_2016_68["optimalPartitions"] if len(p.split(",")) > 1]
complex_partitions_16_set = set(complex_partitions_16)
covered_weight_complex_16 = usewod_df_16.apply(lambda row: row["weight"] if row["set"].issubset(complex_partitions_16_set) else 0, axis=1).sum()
covered_weight_complex_16 / usewod_meta_16["VALID_QUERIES"]

0.2584570188711599

In [15]:
resolved_partitions_16 = []
predicate_map = { int(index): predicate for predicate, index in (line.split("\t") for line in open(f"../../results/queryshapes/usewod_predicate_map.tsv").readlines())}
for partition in complex_partitions_16:
    indices = partition.split(',')
    predicates = ",".join([predicate_map[int(index)] for index in indices])
    resolved_partitions_16.append(predicates)

In [16]:
with open("../../results/queryshapes/usewod_2016_partitions_complex_only.txt", "w") as file:
    for part in resolved_partitions_16:
        file.write(part + "\n")